In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchmetrics import Accuracy
from torchvision import models, transforms
from torchsummary import summary

In [ ]:
def get_abs_path(n_parent: int = 0):
    return Path('../' * n_parent).resolve()

In [ ]:
path = get_abs_path(1)
data_path = path / 'data'
images_paths = data_path.glob('**/*.png')
images_paths = list(images_paths)
images_paths = [str(path) for path in images_paths]

class_names = [d.name for d in data_path.iterdir() if d.is_dir()]
class_labels = {value:key for (key,value) in enumerate(class_names)}
print('Labels:', class_labels)

train_paths, test_paths = train_test_split(images_paths, test_size=0.1, shuffle=True)
val_paths, test_paths = train_test_split(test_paths, test_size=0.5, shuffle=True)
print('train len: %d val len: %d test len: %d' % (len(train_paths), len(val_paths), len(test_paths)))

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((400, 400)),
        transforms.RandomCrop((300, 300)),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validate': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


def get_label_from_path(class_labels, path):
    for name, label in class_labels.items():
        if name in path:
            return label


class CountryDataset(Dataset):
    def __init__(self, images_paths, class_labels, transforms):
        self.images_paths = images_paths
        self.transforms = transforms
        self.labels = []
        for image_path in self.images_paths:
            label = get_label_from_path(class_labels, image_path)
            self.labels.append(label)

    def __len__(self):
        return len(self.images_paths)

    def __getitem__(self, idx):
        path = self.images_paths[idx]
        x = Image.open(path)
        x = self.transforms(x)
        label = self.labels[idx]
        return x, label, path


batch_size = 2
train_dataset = CountryDataset(train_paths, class_labels, data_transforms['train'])
valid_dataset = CountryDataset(valid_paths, class_labels, data_transforms['validate'])
test_dataset = CountryDataset(test_paths, class_labels, data_transforms['test'])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for i in range(3):
    x, label, filename = train_dataset[i]
    name = class_names[label]
    img = mpimg.imread(filename)
    plt.gca().clear()
    plt.title(name)
    plt.imshow(img)
    plt.show()

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {0} device'.format(device))

In [ ]:
# class CountryClassificator(nn.Module):

#     def __init__(self):
#         super(CountryClassificator, self).__init__()

#         self.model = models.resnet50(pretrained=True)

#         self.model.fc = nn.Sequential(  nn.Linear(2048, 512),
#                                         nn.ReLU(),
#                                         nn.Dropout(0.2),
#                                         nn.Linear(512, 10),
#                                         nn.LogSoftmax(dim=1))

#     def forward(self, x):
#         x = self.model(x)
#         return x

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('Using %s device' % device)

In [ ]:
# model = CountryClassificator().cuda()
# model.to(device)

# criterion = nn.NLLLoss()
# optimizer = torch.optim.Adam(model.model.fc.parameters(), lr=0.003)
# summary(model, (3, 224, 224))

In [ ]:
# for x, labels in train_dataloader:
#     print(x)

In [ ]:
# epochs = 10
# for epoch in range(epochs):
#     print('Epoch %d/%d' % (epoch+1, epochs))
#     print('s')
#     total_loss = 0
#     for x, labels in train_dataloader:
#         print(x.shape)
#         x, labels = x.to(device), labels.to(device)
#         optimizer.zero_grad()
#         print('1')
#         preds = model.forward(x)
#         loss = criterion(preds, labels)
#         loss.backward()
#         print('1')
#         optimizer.step()
#         total_loss += loss.item()
#         val_loss = 0
#         val_acc = 0
#         model.eval()
#         print('1')
#         with torch.no_grad():
#             total_val_loss = 0
#             for x, labels in val_dataloader:
#                 x, labels = x.to(device), labels.to(device)
#                 preds = model.forward(x)
#                 val_loss = criterion(preds, labels)
#                 total_val_loss += val_loss.item()
#                 ps = torch.exp(logps)
#                 top_p, top_class = ps.topk(1, dim=1)
#                 equals = top_class == labels.view(*top_class.shape)
#                 accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#                 train_losses.append(running_loss/len(trainloader))
#                 test_losses.append(test_loss/len(testloader))
#                 print(f"Epoch {epoch+1}/{epochs}.. "
#                 f"Train loss: {running_loss/print_every:.3f}.. "
#                 f"Test loss: {test_loss/len(testloader):.3f}.. "
#                 f"Test accuracy: {accuracy/len(testloader):.3f}")
#             running_loss = 0
#             model.train()